In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from glob import glob
import plotly.express as px

pd.options.plotting.backend = 'plotly'

## plotly.io를 import 한 후 renderers 기본값을 꼭 "notebook_connected" 로 설정해주시기 바랍니다.
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
path = 'data/'
files = sorted(glob(path+'*.csv'))

site_info = pd.read_csv(files[4]) # 발전소 정보
energy = pd.read_csv(files[2]) # 발전소별 발전량

dangjin_fcst_data = pd.read_csv(files[0]) # 당진 예보 데이터
dangjin_obs_data = pd.read_csv(files[1]) # 당진 기상 관측 자료

ulsan_fcst_data = pd.read_csv(files[5]) # 울산 예보 데이터
ulsan_obs_data = pd.read_csv(files[6]) # 울산 기상 관측 자료

sample_submission = pd.read_csv(files[3]) # 제출 양식

# 1. 발전소 정보

각 발전소의 위치, 용량, 발전기 각도 정보입니다.

In [3]:
site_info.head()

,Id,Capacity,Address,InstallationAngle,IncidentAngle,Latitude,Longitude
0,당진수상태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
1,당진자재창고태양광,0.7,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
2,당진태양광,1.0,충남 당진시 석문면 교로길 30,30.0,30.0,37.050753,126.510299
3,울산태양광,0.5,울산광역시 남구 용잠로 623,20.0,20.0,35.477651,129.380778


# 2. 발전소별 발전량

2018년 3월 1일부터 2021년 1월 31일까지 1시간 단위로 기록된 각 발전소별 발전량 데이터입니다

In [4]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25632 entries, 0 to 25631
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time               25632 non-null  object 
 1   dangjin_floating   25608 non-null  float64
 2   dangjin_warehouse  25584 non-null  float64
 3   dangjin            25632 non-null  int64  
 4   ulsan              25632 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 1001.4+ KB


In [5]:
energy.head(10)

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00:00,0.0,0.0,0,0
1,2018-03-01 2:00:00,0.0,0.0,0,0
2,2018-03-01 3:00:00,0.0,0.0,0,0
3,2018-03-01 4:00:00,0.0,0.0,0,0
4,2018-03-01 5:00:00,0.0,0.0,0,0
5,2018-03-01 6:00:00,0.0,0.0,0,0
6,2018-03-01 7:00:00,0.0,0.0,0,0
7,2018-03-01 8:00:00,0.0,0.0,0,4
8,2018-03-01 9:00:00,36.0,33.0,37,35
9,2018-03-01 10:00:00,313.0,209.0,318,71


평균적으로 **당진**이 가장 많이 발전합니다. `count`값이 다른 것으로 보아 결측치가 존재함을 알 수 있습니다.

In [6]:
energy.describe()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan
count,25608.000000,25584.000000,25632.000000,25632.000000
mean,122.056311,92.476665,139.653285,64.159761
std,192.041585,146.423366,220.491387,101.020447
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,191.000000,145.000000,227.000000,101.000000
max,753.000000,593.000000,867.000000,392.000000


__당진수상태양광__(24)과 __당진자재창고태양광__(48)에 일부 결측치가 있습니다만 지금은 처리하지 않겠습니다.

In [7]:
energy.isnull().sum()

time                  0
dangjin_floating     24
dangjin_warehouse    48
dangjin               0
ulsan                 0
dtype: int64

`time` 데이터를 살펴보면 새벽 0시가 24:00:00로 표기되어 datetime형으로 바뀌지 않습니다. 이를 조정해주겠습니다.

In [8]:
energy['date'] = energy['time'].apply(lambda x: x.split()[0])
energy['time'] = energy['time'].apply(lambda x: x.split()[1])
energy['time'] = energy['time'].str.rjust(8,'0') # 한자릿수 시간 앞에 0 추가 ex) 3시 -> 03시

# 24시를 00시로 바꿔주기
energy.loc[energy['time']=='24:00:00','time'] = '00:00:00'
energy['time'] = energy['date'] + ' ' + energy['time']
energy['time'] = pd.to_datetime(energy['time'])
energy.loc[energy['time'].dt.hour==0,'time'] += timedelta(days=1)

### 처음 일주일간 발전량 시각화

데이터의 row는 25632개가 있어 약 __일주일 정도의 에너지 발전량__을 시각화해보면 다음과 같습니다. 용량이 큰 __당진__이 대체로 많이 발전하고, 용량이 적은 __울산__이 대체로 적게 발전합니다. 3월 4일 경에는 모든 발전소의 발전량이 낮은데 아무래도 날씨의 영향인 것 같습니다.

In [9]:
fig = px.line(energy[:24*7], x='time', y=['dangjin_floating','dangjin_warehouse','dangjin','ulsan'])
fig.show()

`time` column을 분해하여 의미있을 것 같은 `month`와 `hour`로 시각화해보겠습니다

In [10]:
energy['month'] = energy['time'].dt.month
energy['hour'] = energy['time'].dt.hour

## 월별 발전량

In [11]:
mean_month = energy.groupby('month').mean()
fig = px.bar(mean_month, x=mean_month.index, y=['dangjin_floating','dangjin_warehouse','dangjin','ulsan'])
fig.show()

일반적으로 해가 가장 높게뜨고 기온이 높은 __여름(6월~8월)에 발전량이 가장 많을 것 같지만__ 평균적으론 __봄(3월~5월)이 발전량이 더 높습니다.__ 이는 *태양광판넬의 입사각이 20~30도라는 점, 여름에 강수량이 많다는 점*에서 충분히 이해가 갈 것 같습니다.

## 시간별 발전량

In [12]:
mean_hour = energy.groupby('hour').mean()
fig = px.bar(mean_hour, x=mean_hour.index, y=['dangjin_floating','dangjin_warehouse','dangjin','ulsan'])
fig.show()

지난 3년간 **저녁 8시부터 다음날 아침 06시까지**는 발전량이 없었습니다. 해당 시간은 0으로 초기화해도 될 것 같네요. 오후 1시, 2시가 평균 발전량이 가장 높았습니다.

## 발전소별 발전량 분포

발전량 0을 제외한 데이터의 분포를 보았습니다. 평균적으로 100~200사이 구간 발전량이 많습니다.

In [13]:
cols = ['dangjin_floating','dangjin_warehouse','dangjin','ulsan']
fig = px.box(energy[energy[cols]!=0], x=cols)
fig.update_traces(quartilemethod="exclusive")
fig.show()

# 3. 당진 기상관측자료

당진의 기상관측데이터입니다. 예보 데이터는 본 코드에서 다루지 않겠습니다.

In [14]:
dangjin_obs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25626 entries, 0 to 25625
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지점         25626 non-null  int64  
 1   지점명        25626 non-null  object 
 2   일시         25626 non-null  object 
 3   기온(°C)     25589 non-null  float64
 4   풍속(m/s)    25590 non-null  float64
 5   풍향(16방위)   25590 non-null  float64
 6   습도(%)      25591 non-null  float64
 7   전운량(10분위)  21656 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.6+ MB


In [15]:
dangjin_obs_data.isnull().sum()

지점              0
지점명             0
일시              0
기온(°C)         37
풍속(m/s)        36
풍향(16방위)       36
습도(%)          35
전운량(10분위)    3970
dtype: int64

일부 null값이 보이고 특히 **전운량의 결측치가 많습니다.** 구름없이 맑은 날은 결측치로 처리됨을 예상할 수 있습니다.
- **전운량:** 맑은 날이라고 가정, 0으로 fill
- **기온, 풍속, 풍향, 습도:** 바로 앞 시간대의 관측량으로 대체, ffill

In [16]:
def weather_preprocessing(df):
    df['전운량(10분위)'].fillna(0, inplace=True)
    df.fillna(method='ffill', inplace=True)
    df['일시'] =  pd.to_datetime(df['일시'])
    return df

In [17]:
dangjin_obs_data = weather_preprocessing(dangjin_obs_data)
dangjin_obs_data.isnull().sum()

지점           0
지점명          0
일시           0
기온(°C)       0
풍속(m/s)      0
풍향(16방위)     0
습도(%)        0
전운량(10분위)    0
dtype: int64

## 발전량과 날씨의 상관관계 (당진)

In [18]:
df_joined = pd.merge(energy, dangjin_obs_data, left_on='time', right_on='일시', how='inner')
cols = ['time','dangjin_floating', 'dangjin_warehouse', 'dangjin', '기온(°C)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '전운량(10분위)']
dangjin = df_joined[cols]

In [19]:
dangjin.corr() # 날씨와의 상관계수

,dangjin_floating,dangjin_warehouse,dangjin,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
dangjin_floating,1.000000,0.961408,0.974142,0.307088,0.438443,0.344152,-0.632884,-0.188188
dangjin_warehouse,0.961408,1.000000,0.972938,0.311508,0.426527,0.339161,-0.637108,-0.168499
dangjin,0.974142,0.972938,1.000000,0.294323,0.435736,0.348794,-0.644704,-0.185427
기온(°C),0.307088,0.311508,0.294323,1.000000,0.168179,0.128834,-0.000126,0.037965
풍속(m/s),0.438443,0.426527,0.435736,0.168179,1.000000,0.534637,-0.416005,0.006391
풍향(16방위),0.344152,0.339161,0.348794,0.128834,0.534637,1.000000,-0.371380,-0.004386
습도(%),-0.632884,-0.637108,-0.644704,-0.000126,-0.416005,-0.371380,1.000000,0.176116
전운량(10분위),-0.188188,-0.168499,-0.185427,0.037965,0.006391,-0.004386,0.176116,1.000000


In [20]:
fig = px.imshow(dangjin.corr())
fig.show()

**기온은 0.3, 풍속은 0.43, 풍향은 0.4 정도의 양의 상관관계**를 보이고 있으며, **습도 -0.63, 전운량 -0.18 정도의 음의 상관관계**를 보이고 있습니다. 절댓값으로만 보자면 **습도**가 가장 상관관계가 뚜렷합니다.

# 4. 울산 기상관측자료

울산의 기상관측데이터입니다. 예보 데이터는 본 코드에서 다루지 않겠습니다.

In [21]:
ulsan_obs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25632 entries, 0 to 25631
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지점         25632 non-null  int64  
 1   지점명        25632 non-null  object 
 2   일시         25632 non-null  object 
 3   기온(°C)     25628 non-null  float64
 4   풍속(m/s)    25631 non-null  float64
 5   풍향(16방위)   25631 non-null  float64
 6   습도(%)      25631 non-null  float64
 7   전운량(10분위)  24807 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.6+ MB


In [22]:
ulsan_obs_data.isnull().sum()

지점             0
지점명            0
일시             0
기온(°C)         4
풍속(m/s)        1
풍향(16방위)       1
습도(%)          1
전운량(10분위)    825
dtype: int64

In [23]:
ulsan_obs_data = weather_preprocessing(ulsan_obs_data)

울산은 당진보다 상대적으로 적은 결측치를 가지고 있습니다. 같은 방법으로 전처리해주겠습니다.

## 발전량과 날씨의 상관관계 (당진)

In [24]:
df_joined = pd.merge(energy, ulsan_obs_data, left_on='time', right_on='일시', how='inner')
cols = ['time','ulsan', '기온(°C)', '풍속(m/s)', '풍향(16방위)', '습도(%)', '전운량(10분위)']
ulsan = df_joined[cols]

In [25]:
ulsan.corr() # 날씨와의 상관계수

,ulsan,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위)
ulsan,1.000000,0.240879,0.307515,-0.186768,-0.447506,-0.190440
기온(°C),0.240879,1.000000,-0.070631,-0.293445,0.355624,0.203843
풍속(m/s),0.307515,-0.070631,1.000000,0.172195,-0.300215,-0.014533
풍향(16방위),-0.186768,-0.293445,0.172195,1.000000,-0.097894,-0.057273
습도(%),-0.447506,0.355624,-0.300215,-0.097894,1.000000,0.448958
전운량(10분위),-0.190440,0.203843,-0.014533,-0.057273,0.448958,1.000000


In [26]:
fig = px.imshow(ulsan.corr())
fig.show()

울산은 당진과 다르게 **풍향**에서 음의 상관관계를 보이고 있습니다. 상관계수의 절댓값이 그리 크지않은 값들이라 유의미하다고는 보기 힘들겠습니다.

# 5. 제출 양식

In [27]:
sample_submission.head()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0,0,0,0
1,2021-02-01 02:00:00,0,0,0,0
2,2021-02-01 03:00:00,0,0,0,0
3,2021-02-01 04:00:00,0,0,0,0
4,2021-02-01 05:00:00,0,0,0,0


In [28]:
sample_submission.tail()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1387,2021-07-08 20:00:00,0,0,0,0
1388,2021-07-08 21:00:00,0,0,0,0
1389,2021-07-08 22:00:00,0,0,0,0
1390,2021-07-08 23:00:00,0,0,0,0
1391,2021-07-08 24:00:00,0,0,0,0


제출양식은 **2021년 2월 1일부터 7월 8일까지**로 이루어져있습니다. 위 데이터를 학습한 모델을 통해 예측을 진행하면 되겠습니다. 다만, **최솟값은 0, 최댓값은 해당 발전소의 발전용량이 되도록 조정을 해줘야합니다.** 또한 본 대회의 평가산식 NMAE-10에 따르면 **발전용량의 10%이상 발전된 데이터만으로 평가**합니다. 이를 유의하셔야겠습니다.

private 평가기간에는 매일매일 새로운 결과를 제출할 수 있습니다. 해당 일자의 예보데이터는 그 전날 수집이 가능하기때문에, 날씨 데이터를 이용하실 분들은 api를 이용해 다음 날 데이터를 가져오시는 게 좋습니다. [DACON.Dobby님의 코드](https://dacon.io/competitions/official/235720/codeshare/2555?page=1&dtype=recent)로 쉽게 받아올 수 있습니다.